In [23]:
# pip install --user albumentations

## Import

In [24]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Settting

In [25]:
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 5,
    'LEARNING_RATE': 3e-4,
    'BATCH_SIZE': 32,
    'SEED': 41
}

이미지의 크기를 가로와 세로 모두 224 픽셀로 설정

epochs : 데이터셋을 몇 번 반복하여 모델을 학습시킬지를 정하는 값

학습률(learning rate)은 모델의 가중치를 얼마나 크게 조정할지를 결정하는 파라미터

BATCH_SIZE : 한 번에 학습하는 데이터의 수, 즉 배치 크기를 의미

난수 생성 시 사용할 시드(seed) 값

### -> 딥러닝 모델 학습에 사용될 여러 설정값을 포함

## Fixed RandomSeed

In [26]:
def seed_everything(seed):

    ###random module의 시드 고정
    random.seed(seed) 

    #해시 함수의 랜덤성 제어, 자료구조 실행할 때 동일한 순서 고정
    os.environ['PYTHONHASHSEED'] = str(seed) 

    #numpy 랜덤 숫자 일정
    np.random.seed(seed) 

    # torch라이브러리에서 cpu 텐서 생성 랜덤 시드 고정
    torch.manual_seed(seed) 

    # cuda의 gpu텐서에 대한 시드 고정
    torch.cuda.manual_seed(seed) 

     # 백엔드가 결정적 알고리즘만 사용하도록 고정 
    torch.backends.cudnn.deterministic = True 

    # CuDNN이 여러 내부 휴리스틱을 사용하여 가장 빠른 알고리즘 동적으로 찾도록 설정
    torch.backends.cudnn.benchmark = True 

np.random.seed(seed) - NumPy 라이브러리의 랜덤 함수들이 일관된 결과를 반환하도록 시드를 설정

torch.manual_seed(seed) 및 torch.cuda.manual_seed(seed) - PyTorch의 CPU 및 GPU 연산에 대해 랜덤 시드를 설정하여, 텐서 연산의 결과가 일관되게 유지

### -> 다양한 라이브러리에 걸쳐 랜덤 시드를 고정하여 실험의 재현성을 보장하기 위한 함수이다.

## Train & Validation Split

In [27]:
df = pd.read_csv('./train.csv')
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])


### -> 전체 데이터 세트를 훈련 세트(train)와 검증 세트(val)로 분리


stratify=df['label'] 레이블의 분포를 균등하게 유지하며 데이터 나누기

random_state=CFG['SEED'] : 데이터 분할시 랜덤성의 일관성을 유지하기 위해 설정된 시드


## Label-Encoding

In [28]:
le = preprocessing.LabelEncoder() # 라벨인코딩 /라벨(목표 변수)를 정수로 인코딩
# train, label의 라벨인코딩 과정 진행
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

### -> 라벨 인코딩 실행

## CustomDataset

In [29]:

class CustomDataset(Dataset):
## 파일 경로와 라벨을 받아, 데이터를 로드하고 전처리하는 데이터셋 생성성
#img_path_list: 이미지 파일들의 경로를 포함하는 리스트 
#label_list: 각 이미지에 해당하는 라벨을 포함하는 리스트
#transforms: 이미지에 적용할 변환을 정의한 객체
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        # 이미지 읽어오기
        image = cv2.imread(img_path)

        #변환 함수가 제공되면, 변환 함수를 이미지에 적용합니다. 이는 데이터 증강이나 전처리 과정을 의미한다
        #변환함수: from albumentations.pytorch.transforms import ToTensorV2 에서 임포트
        if self.transforms is not None:
            image = self.transforms(image = image)['image']

        # 라벨이 있다면 이미지와 함께 라벨 반환
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        
        # 라벨(학습할 이미지)이 없다면 이미지만 반환환
        else:
            return image
    
    #이미지 경로 목록의 길이를 반환
    def __len__(self):
        return len(self.img_path_list)

# Compose는 여러 변환을 연속적으로적용할 수 있게 해주는 함수. (IMG 사이즈 224로 설정되어 있음)
# 이미지 크기조정, 정구화, 텐서로 변환 포함.
'''
Normalize(mean=0.485, 0.456, 0.406값은 각 채널별 평균)
std=(0.229, 0.224, 0.225 값은 각 채널별 표준편차)
max_pixel_value: 이미지의 최대 픽셀 값 (8비트의 경우 255가 최대값)
always_apply= Ture: 변환이 데이터셋의 모든 이미지에 대해 항상 적용.
p: 변환이 적용될 확률: (0~1 사이)
대부분의 경우 always_apply=True로 하고 p를 조절해서 사용 
'''
train_transform = A.Compose([

                            #이미지를 CFG 딕셔너리에 정의된 크기(IMG_SIZE)로 크기 조정
                            #모든 이미지를 224x224 픽셀의 크기로 조절
                            A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),

                            #이미지의 픽셀 값을 정규화
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),

                            #이미지 데이터를 PyTorch 텐서로 변환
                            ToTensorV2()])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()])

## train데이터셋 설정 및 불러오기
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## val 데이터셋 설정 및 불러오기
#검증 데이터셋을 위한 CustomDataset 객체를 생성
val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


CustomDataset 만들기

1. 클래스를 상속받는다
self, img_path_list, label_list 를 불러온다
2. 데이터를 로드하고 이미지를 불러온다

3. 변환함수로 이미지 전처리를 진행

    3.1 정답 이미지가 있다면 이미지와 함께 반환 
    
    3.2 정답 이미지가 없다면 이미지만 반환

4. 이미지 경로 목록의 길이를 반환

5. 훈련데이터를 compose  한다

    5.1 이미지 크기 조절

    5.2 이미지의 픽셀 정규화

    5.3 이미지 데이터 pytorch 텐서로 변화


6. 테스트 데이터도 같은 절차

***변환함수 : from albumentations.pytorch.transforms import ToTensorV2에서 임포트

### -> 데이터를 로드하고 이미지 전처리과정을 저장한 CustomDataset 함수를 정의한다 , 훈련데이터를 compose 한다

## Model Define

In [30]:
# PyTorch의 신경망 모듈인 nn.Module을 상속받아 BaseModel이라는 새로운 클래스를 정의
class BaseModel(nn.Module):

    #클래스의 생성자 함수로, 객체가 생성될 때 자동으로 호출
    def __init__(self, num_classes=len(le.classes_)):

        #베이스의 부모클래스인 nn.Module의 생성자 함수를 호출
        super(BaseModel, self).__init__()

        # models.efficientnet_b0를 사용하여 모델의 백본(기본 구조)을 설정
        self.backbone = models.efficientnet_b0(pretrained=True)

        # 백본 모델의 출력을 받아 최종적으로 클래스 수에 맞는 출력을 생성하는 선형 분류기를 정의 , 기본 출력크기 1,000으로 정의
        self.classifier = nn.Linear(1000, num_classes)

    #모델의 순전파 함수를 정의
    def forward(self, x):

        ##입력 데이터 x를 백본 모델에 통과시켜 특성을 추출
        x = self.backbone(x) 

        ## 추출된 특성을 선형 분류기에 전달하여 최종 출력을 생성
        x = self.classifier(x) 
        return x

### -> BaseModel이라는 이름의 파이썬 클래스를 정의하는 부분으로, PyTorch의 nn.Module을 상속받아 딥러닝 모델을 구성

1.nn.Model 을 상속받아 basemodel이라는 새로운 클래스 만들기

    1-1.생성자 함수 만들기

        1-1-1.super로 basemodel 의 부모클래스 호출

        1-1-2.모델의 백본구조 설정

        1-1-3.클래스 수에 맞게 출력을 생성하는 선형분류기 만들기

    1-2 모델의 순전파함수 만들기
        1-2-1 입력데이터를 백본모델에 적용하여 특성추출
    
        1-2-2 추출된 특성을 선형분류기에 전달하여 최종 출력 생성

*** 백본의 역할:

특성 추출: 백본은 입력 이미지에서 유용한 정보를 추출하는 데 중점을 둔다. 이 정보는 이미지의 시각적 패턴이나 객체의 형태와 같은 중요한 데이터를 포함할 수 있다.

다양한 작업 기반: 추출된 특성은 다른 신경망의 입력으로 사용될 수 있다. 예를 들어, 객체 인식이나 분할을 위한 추가 네트워크 레이어에 연결될 수 있다.

In [2]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):

    ## 모델을 해당 디바이스로 옮김(cpu, gpu)
    model.to(device) 

    ## 손실함수 정의하고 해당 device로 옮김
    criterion = nn.CrossEntropyLoss().to(device) 
    
    # 성능 기록 초기화
    best_score = 0
    best_model = None
    
    # 설정한 하이퍼파라미터의 epochs만큼 반복
    for epoch in range(1, CFG['EPOCHS']+1):

        ##모델을 훈련모드로 설정
        model.train() 
        train_loss = []

        # 반복을 통해서 배치 단위로 이미지와 라벨을 가져옴
        for imgs, labels in tqdm(iter(train_loader)):

            # 이미지를 실수형으로 변경한 후 device로 올림
            imgs = imgs.float().to(device)

            # 데이터 타입 long으로 변경한 후 device로 올림 (int로 변경하였을 때, error 발생했음)
            labels = labels.long().to(device)  
            
            # 이전 그레디언트가 누적될 가능성이 있으니 초기화
            optimizer.zero_grad() 
            
            # 모델의 이미지를 입력하여 출력을 얻음
            output = model(imgs) 

            # 손실 함수를 통해 손실 값을 계산함.
            loss = criterion(output, labels) 
            
            # 손실에 대한 그레디언트 계산
            loss.backward() 

            # 옵티마이저를 통해 모델의 가중치 업데이트
            optimizer.step() 
            
            # loss.item()은 현재 배치에 대한 손실 값을 파이썬의 floate 타입으로 변환
            train_loss.append(loss.item()) 
            # 훈련 과정에서 각 배치를 처리할 때마다 이 줄이 실행되어, 각 배치의 손실 값을 train_loss 리스트에 순차적으로 추가
                    
        # 각 에포크마다 validation함수를 호출하여서 검증 세트에서 모델의 성능을 평가
        _val_loss, _val_score = validation(model, criterion, val_loader, device)

        # 각 배치에서 계산된 모든 손실 값의 평균을 구함
        _train_loss = np.mean(train_loss) 
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 Score : [{_val_score:.5f}]')

        # scheduler이 설정되어 있다면 검증 성능에 따라 학습률을 조정
        if scheduler is not None:
            scheduler.step(_val_score)
            
        # 가장 좋은 성능을 보인 모델을 반환
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

def validation(model, criterion, val_loader, device):
    model.eval() # 평가모드
    val_loss = []
    preds, true_labels = [], []

    # 평가모드의 경우에는 gradient를 초기화하는 부분이 없음 (backward 필요없음. 오직 평가만!)
    # 이 블록 내에서 그레디언트 계산을 중단하여, 필요하지 않은 메모리 사용을 줄이고 계산 속도 향상.
    with torch.no_grad(): 
        
        #검증 데이터 로더에서 배치 단위로 이미지와 라벨을 가져온다. tqdm은 진행 상태를 시각적으로 표시한다
        for imgs, labels in tqdm(iter(val_loader)):

            #이미지 데이터를 실수형으로 변환하고 계산을 수행할 디바이스(CPU 또는 GPU)로 이동한다
            imgs = imgs.float().to(device)

            # 데이터 타입 long으로 변경한 후 device로 올림 (int로 변경하였을 때, error 발생했음)
            labels = labels.long().to(device)  
            
            #변환된 이미지 데이터를 모델에 입력하여 예측 결과를 얻는다
            pred = model(imgs)
            
            #손실 함수를 사용해 모델의 예측과 실제 라벨 간의 손실을 계산한다.
            loss = criterion(pred, labels)
            
            # pred는 모델이 반환한 예측값. 각 클래스에 대한 확률 또는 점수를 포함하는 텐서. argmax(1)은 각 샘플에 대해 가장 높은 점수를 가진 클래스의 인덱스를 찾아줌.
            # detach()는 현재 계산 그래프로부터 이 텐서를 분리하여, 이후 연산이 그래프에 기록되지 않도록함. 메모리 사용량 줄임
            # cpu()는 cpu로 옮김 (GPU에 있었다면)
            # numpy()는 텐서를 numpy 배열로 변환
            # tolist()는 numpy 배열을 파이썬 리스트로 변환
            preds += pred.argmax(1).detach().cpu().numpy().tolist()

            # 실제 라벨도 위와 동일한 과정 진행
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        # average = 'macro'는 F1점수를 계산할 때, 각 클래스에 대한 F1점수를 동일한 가중치로 평균내어 전체 클래스에 대한 평균 F1점수를 계산.
        # 각 클래스의 샘플 크기와 관계없이 모든 클래스를 동등하게 취급. 이는 클래스 불균형이 있을 때 유용하며, 모든 클래스를 공평하게 평가하고자 할 때 사용.
        _val_score = f1_score(true_labels, preds, average='macro')
    
    return _val_loss, _val_score


1. train 이라는 클래스를 만든다

	1-1. 모델을 해당 디바이스로 옮긴다
	
	1-2. 손실함수를정의하고 해당 디바이스로 옮긴다

	1-3 성능기록을 초기화 한다

	1-4 설정한 하이퍼파라미터단계에서의 epochs만큼 반복

		1-4-1 모델을 훈련모드로 설정
		
		1-4-2 반복을 통해서 이미지와 라벨을 가져온다

			1-4-2-1 이미지를 실수형으로 변경한다

			1-4-2-2 데이터 타입을 long으로 변경한다

			1-4-2-3 기울기를 초기화

			1-4-2-4 모델의 이미지에 출력을 얻는다

			1-4-2-5 손실함수를 통해 손실값을 계산한다

			1-4-2-6 손실에 대한 기울기를 계산한다

			1-4-2-7 기울기에 대한 가중치를 업데이트한다

			1-4-2-8 손실값을 float으로 변환
		
		1-4-3 각 에포크마다 validation함수를 호출하여 모델 성능을 평가

		1-4-4 각 배치에서 계산된 손실값의 평균을 구한다

		1-4-5  scheduler에 따라 학습률 조정

		1-4-6 가장 좋은 모델로 반환

2. validation 함수 만들기
	
	2.1 평가모드로 만들기
	
	2.2 블록내 기울기 계산 중단

		2.2.1 검증 데이터에서 배치단위로 이미지와 라벨을 가져오기 

		2.2.2 데이터 타입을 long으로 변환

		2.2.3 이미지를 예측하여 결과를 얻는다

		2.2.4 손실함수를 사용하여 예측과 실제간의 손실을 계산

		2.2.5 예측 결과를 처리하고, 최종적으로 파이썬 리스트에 추가

		2.2.6 실제라벨도 처리하고 리스트에 추가 

		2.2.7 각 배치의 손실값을 val_loss라는 리스트에 추가

	2.3 val_loss 리스트에 저장된 모든 검증 손실값들의 평균을 계산

	2.4 예측 성능을 평가하기 위해 F1 점수를 계산

*** scheduler : scheduler는 학습률 스케줄러(Learning Rate Scheduler)를 의미. 이는 딥러닝 모델을 훈련할 때 학습률(learning rate)을 동적으로 조정하는 도구

***gradient = 기울기

***"모델을 해당 디바이스로 옮긴다"라는 표현은 PyTorch에서 모델이 연산을 수행할 하드웨어를 선택하는 것을 의미 

***torch.long은 파이토치에서 정수형 데이터를 나타내는 데이터 타입이며, 64비트의 정수 값을 가짐

### -> train 과 validation 함수로 훈련 방법과 평가 방법을 정의 

In [34]:
# 모델은 basemodel 가져옴
model = BaseModel()

#평가모드로 전환 
model.eval() 

## optimizer 'adam'으로 설정 / 학습률 위의 하이퍼파라미터
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"]) 

#학습률을 동적으로 조정하는 스케줄러 설정. 검증 성능이 개선되지 않으면 학습률 감소.
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

#train 함수를 호출하여 모델 학습 진행
infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|██████████| 149/149 [00:18<00:00,  8.22it/s]


Epoch [1], Train Loss : [1.05827] Val Loss : [0.53192] Val F1 Score : [0.84296]


100%|██████████| 149/149 [00:09<00:00, 15.38it/s]


Epoch [2], Train Loss : [0.29028] Val Loss : [0.52310] Val F1 Score : [0.85927]


100%|██████████| 149/149 [00:10<00:00, 14.52it/s]


Epoch [3], Train Loss : [0.13385] Val Loss : [0.56164] Val F1 Score : [0.86317]


100%|██████████| 149/149 [00:10<00:00, 14.24it/s]


Epoch [4], Train Loss : [0.11459] Val Loss : [0.68529] Val F1 Score : [0.84690]


100%|██████████| 149/149 [00:10<00:00, 14.41it/s]

Epoch [5], Train Loss : [0.11972] Val Loss : [0.64424] Val F1 Score : [0.85878]


1. 정의한 basemodel을 가져온다

2. 평가모드로 전환 

3. optimizer를 adam 으로 설정 

4. schduler 설정 

5. 훈련함수를 호출하여 모델 훈련하기

***평가모드에서는 모델의 매개변수가 업데이트되지 않으며, 그레디언트도 계산되거나 저장되지 않는다.

*** Adam은 "Adaptive Moment Estimation"의 약자로, 딥러닝에서 널리 사용되는 최적화 알고리즘 중 하나이다. 

특징으로는 1. 적응적 학습률 2. 모맨텀과 RMSprop의 결합 3. 효율성 4. 하이퍼파라미터

***train_loader는 파이토치(PyTorch)에서 사용하는 데이터 로더 중 하나로, 모델 훈련에 사용되는 훈련 데이터셋을 배치 단위로 모델에 공급하는 역할

### -> 훈련함수를 호출하여 훈련을 진행

In [35]:
test = pd.read_csv('./test.csv')

#CustomDataset은 사용자 정의 데이터셋 클래스. 
#여기서는 테스트 이미지의 경로가 포함된 test['img_path'].values와 데이터에 적용할 변환 test_transform을 매개변수로 사용. 라벨 정보는 제공되지 않으므로 None으로 설정
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)

#이 로더는 테스트 데이터셋을 모델에 공급하는 데 사용
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

#inference 함수는 모델, 데이터 로더, 그리고 실행할 디바이스를 매개변수로 받아 모델의 추론을 수행
def inference(model, test_loader, device):

    #모델을 평가 모드로 설정하여, 모든 훈련 특정 기능(예: 드롭아웃)을 비활성화
    model.eval()
    preds = []

    #이 블록은 그레디언트 계산을 비활성화
    with torch.no_grad(): 

        #tqdm을 사용하여 진행 상태를 시각적으로 표시하면서 test_loader에서 이미지 배치를 순차적으로 로드
        for imgs in tqdm(iter(test_loader)):

            #로드된 이미지를 실수형으로 변환하고 계산을 수행할 디바이스로 이동
            imgs = imgs.float().to(device)
            
            #변환된 이미지 배치를 모델에 입력하여 예측 결과
            pred = model(imgs)
            
            #모델의 출력에서 가장 확률이 높은 클래스의 인덱스를 추출하고, 이를 리스트에 추가
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    #원래의 문자열 또는 범주형 레이블로 변환
    preds = le.inverse_transform(preds)
    return preds

#inference 함수를 호출하여 infer_model, test_loader, device를 사용해 추론을 수행
preds = inference(infer_model, test_loader, device)

100%|██████████| 213/213 [00:15<00:00, 13.69it/s]


1. 테스트 파일 불러오기 

2. 정의한 CustomDatset 을 불러온다

3. DataLoader을 불러온다

4. inference 함수를 정의한다

	4.1 모델을 평가모드로 전환한다

	4.2 블록을 기울기 계산을 비활성화 한다

		4.2.1 for 함수를 통해 test loader 이미지를 순차적으로 로드한다

		4.2.2 이미지를 실수형으로 변환한다

		4.2.3 변환된 이미지를 모델에 입력하여 예측한다

	4.3 inverse_transform을 사용하여 수치형 레이블을 범주형으로 원래의 범주형으로 변환

5. infernce 함수를 호출하여 추론을 수행

***가장 확률이 높은 클래스의 인덱스 : 

클래스 : 각 이미지

확률이 높은 이미지 : 정답일 확률 높은 이미지

***inverse_transform 메소드는 이러한 변환을 역으로 수행하여, 모델의 출력으로 얻은 예측된 정수 인덱스를 원래의 범주형 레이블로 되돌린다.

### -> 추론 함수를 정의하고 추론을 진행

In [36]:
submit = pd.read_csv('./sample_submission.csv')
submit['label'] = preds
submit.to_csv('./baseline_submit.csv', index=False)

# 전체 프로세스

1. 하이퍼 파라미터 설정을 통하여 모델 학습에 사용될 설정값들을 세팅한다

2. 랜덤 시드를 고정하여 재현성을 보장한다 

3. 전체 데이터에서 훈련 세트와 검증세트로 분리한다

4. 라벨 인코딩을 실행한다

5. 데이터를 로드, 전처리하는 CustomDataset 함수를 정의한다

6. BaseModel이라는 함수를 정의하여 모델링을 한다

7. train 과 validation 함수를 정의하여 훈련방법과 평가방법을 정의한다

8. train함수를 호출하여 훈련을 진행한다

9. inference 함수를 정의하고 추론을 진행한다